# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [6]:
import numpy as np
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    words = [word for word in text]
    vocab = Counter(words)
    vocab = sorted(vocab, key = vocab.get, reverse =False)
    vocab_to_int = {word:ii for ii, word in enumerate(vocab, 0)}
    int_to_vocab = {ii:word for ii, word in enumerate(vocab, 0)}
    return vocab_to_int, int_to_vocab

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [7]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    punctuations = {'.':"||period||", ',':"||comma||",'"':"||quotation_mark||",
                    ';':"||semicolon||",'!':"||exclamation||",'?':"||question_mark||",
                    '(':"||left_parentheses||",')':"||right_parentheses||",
                    '--':"||dash||",'\n':"||return||" }
    return punctuations

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [8]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [9]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [10]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.3'), 'Please use TensorFlow version 1.3 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.3.0
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [11]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    inputs = tf.placeholder(tf.int32,[None,None], name="input")
    targets = tf.placeholder(tf.int32, [None, None], name="targets")
    learning_rate = tf.placeholder(tf.float32, name= "learning_rate")
    return inputs, targets, learning_rate

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [13]:
def build_cell(rnn_size):
  return tf.contrib.rnn.BasicLSTMCell(rnn_size, state_is_tuple=True)

def get_init_cell(batch_size, rnn_size, rnn_layers=2):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(rnn_size) for _ in range(rnn_layers)])
    zero_state = cell.zero_state(batch_size, tf.float32)
    initial_state = tf.identity(zero_state,name="initial_state")
    return cell, initial_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [14]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    embedding = tf.Variable(tf.random_uniform([vocab_size,embed_dim] ,-1.0, 1.0))
    embed = tf.nn.embedding_lookup(embedding,input_data)                       
    return embed

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [15]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    outputs, final_state1 = tf.nn.dynamic_rnn(cell,inputs, dtype= tf.float32)
    final_state = tf.identity(final_state1, name= "final_state")
    return outputs, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [16]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    embed = get_embed(input_data, vocab_size, embed_dim)
    outputs, final_state = build_rnn(cell, embed)
    logits = tf.contrib.layers.fully_connected(outputs,num_outputs= vocab_size,activation_fn=None)
    return logits, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For example, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [17]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    n_words_per_batch =  batch_size * seq_length
    n_batches = len(int_text)//n_words_per_batch
    int_text =np.array(int_text)
    int_input = int_text[:n_words_per_batch * n_batches]
    int_targets = int_text[1:n_words_per_batch * n_batches]
    int_targets=np.append(int_targets,int_input[0])
    int_input = np.reshape(int_input,(batch_size,-1))
    int_targets = np.reshape(int_targets,(batch_size,-1))
    batches =[]
    for n in range(0,int_input.shape[1],seq_length):
      batch=[]
      x = int_input[:,n:n + seq_length]
      y = int_targets[:,n:n + seq_length]
      batch.append(x)
      batch.append(y)
      batches.append(batch)
    batches=np.array(batches)
    return batches


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [24]:
# Number of Epochs
num_epochs = 70
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 256
# Embedding Dimension Size
embed_dim = 256
# Sequence Length
seq_length = 5
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = 2

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [25]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [26]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/107   train_loss = 8.822
Epoch   0 Batch    2/107   train_loss = 8.249
Epoch   0 Batch    4/107   train_loss = 6.923
Epoch   0 Batch    6/107   train_loss = 7.009
Epoch   0 Batch    8/107   train_loss = 7.282
Epoch   0 Batch   10/107   train_loss = 6.852
Epoch   0 Batch   12/107   train_loss = 7.065
Epoch   0 Batch   14/107   train_loss = 6.840
Epoch   0 Batch   16/107   train_loss = 7.127
Epoch   0 Batch   18/107   train_loss = 6.951
Epoch   0 Batch   20/107   train_loss = 7.062
Epoch   0 Batch   22/107   train_loss = 6.986
Epoch   0 Batch   24/107   train_loss = 6.878
Epoch   0 Batch   26/107   train_loss = 6.528
Epoch   0 Batch   28/107   train_loss = 5.784
Epoch   0 Batch   30/107   train_loss = 5.746
Epoch   0 Batch   32/107   train_loss = 5.869
Epoch   0 Batch   34/107   train_loss = 5.957
Epoch   0 Batch   36/107   train_loss = 5.672
Epoch   0 Batch   38/107   train_loss = 5.799
Epoch   0 Batch   40/107   train_loss = 6.022
Epoch   0 Batch   42/107   train_l

Epoch   3 Batch   47/107   train_loss = 4.373
Epoch   3 Batch   49/107   train_loss = 4.423
Epoch   3 Batch   51/107   train_loss = 4.468
Epoch   3 Batch   53/107   train_loss = 4.344
Epoch   3 Batch   55/107   train_loss = 4.470
Epoch   3 Batch   57/107   train_loss = 4.204
Epoch   3 Batch   59/107   train_loss = 4.365
Epoch   3 Batch   61/107   train_loss = 4.352
Epoch   3 Batch   63/107   train_loss = 4.215
Epoch   3 Batch   65/107   train_loss = 4.337
Epoch   3 Batch   67/107   train_loss = 4.302
Epoch   3 Batch   69/107   train_loss = 4.251
Epoch   3 Batch   71/107   train_loss = 4.385
Epoch   3 Batch   73/107   train_loss = 4.525
Epoch   3 Batch   75/107   train_loss = 4.124
Epoch   3 Batch   77/107   train_loss = 4.315
Epoch   3 Batch   79/107   train_loss = 4.226
Epoch   3 Batch   81/107   train_loss = 4.259
Epoch   3 Batch   83/107   train_loss = 4.044
Epoch   3 Batch   85/107   train_loss = 4.167
Epoch   3 Batch   87/107   train_loss = 4.243
Epoch   3 Batch   89/107   train_l

Epoch   6 Batch   86/107   train_loss = 3.562
Epoch   6 Batch   88/107   train_loss = 3.812
Epoch   6 Batch   90/107   train_loss = 3.499
Epoch   6 Batch   92/107   train_loss = 3.733
Epoch   6 Batch   94/107   train_loss = 3.575
Epoch   6 Batch   96/107   train_loss = 3.629
Epoch   6 Batch   98/107   train_loss = 3.613
Epoch   6 Batch  100/107   train_loss = 3.558
Epoch   6 Batch  102/107   train_loss = 3.617
Epoch   6 Batch  104/107   train_loss = 3.470
Epoch   6 Batch  106/107   train_loss = 3.539
Epoch   7 Batch    1/107   train_loss = 3.664
Epoch   7 Batch    3/107   train_loss = 3.688
Epoch   7 Batch    5/107   train_loss = 3.636
Epoch   7 Batch    7/107   train_loss = 3.564
Epoch   7 Batch    9/107   train_loss = 3.562
Epoch   7 Batch   11/107   train_loss = 3.756
Epoch   7 Batch   13/107   train_loss = 3.524
Epoch   7 Batch   15/107   train_loss = 3.741
Epoch   7 Batch   17/107   train_loss = 3.742
Epoch   7 Batch   19/107   train_loss = 3.645
Epoch   7 Batch   21/107   train_l

Epoch  10 Batch   18/107   train_loss = 3.057
Epoch  10 Batch   20/107   train_loss = 3.105
Epoch  10 Batch   22/107   train_loss = 3.244
Epoch  10 Batch   24/107   train_loss = 3.035
Epoch  10 Batch   26/107   train_loss = 3.119
Epoch  10 Batch   28/107   train_loss = 2.925
Epoch  10 Batch   30/107   train_loss = 2.924
Epoch  10 Batch   32/107   train_loss = 2.871
Epoch  10 Batch   34/107   train_loss = 3.075
Epoch  10 Batch   36/107   train_loss = 2.957
Epoch  10 Batch   38/107   train_loss = 2.943
Epoch  10 Batch   40/107   train_loss = 3.107
Epoch  10 Batch   42/107   train_loss = 3.164
Epoch  10 Batch   44/107   train_loss = 2.910
Epoch  10 Batch   46/107   train_loss = 2.990
Epoch  10 Batch   48/107   train_loss = 3.071
Epoch  10 Batch   50/107   train_loss = 2.883
Epoch  10 Batch   52/107   train_loss = 3.086
Epoch  10 Batch   54/107   train_loss = 3.092
Epoch  10 Batch   56/107   train_loss = 3.008
Epoch  10 Batch   58/107   train_loss = 2.993
Epoch  10 Batch   60/107   train_l

Epoch  13 Batch   57/107   train_loss = 2.464
Epoch  13 Batch   59/107   train_loss = 2.658
Epoch  13 Batch   61/107   train_loss = 2.637
Epoch  13 Batch   63/107   train_loss = 2.471
Epoch  13 Batch   65/107   train_loss = 2.565
Epoch  13 Batch   67/107   train_loss = 2.595
Epoch  13 Batch   69/107   train_loss = 2.546
Epoch  13 Batch   71/107   train_loss = 2.663
Epoch  13 Batch   73/107   train_loss = 2.677
Epoch  13 Batch   75/107   train_loss = 2.428
Epoch  13 Batch   77/107   train_loss = 2.575
Epoch  13 Batch   79/107   train_loss = 2.608
Epoch  13 Batch   81/107   train_loss = 2.590
Epoch  13 Batch   83/107   train_loss = 2.560
Epoch  13 Batch   85/107   train_loss = 2.563
Epoch  13 Batch   87/107   train_loss = 2.626
Epoch  13 Batch   89/107   train_loss = 2.547
Epoch  13 Batch   91/107   train_loss = 2.490
Epoch  13 Batch   93/107   train_loss = 2.495
Epoch  13 Batch   95/107   train_loss = 2.457
Epoch  13 Batch   97/107   train_loss = 2.596
Epoch  13 Batch   99/107   train_l

...

Epoch  53 Batch   97/107   train_loss = 1.310
Epoch  53 Batch   99/107   train_loss = 1.180
Epoch  53 Batch  101/107   train_loss = 1.193
Epoch  53 Batch  103/107   train_loss = 1.149
Epoch  53 Batch  105/107   train_loss = 1.160
Epoch  54 Batch    0/107   train_loss = 1.189
Epoch  54 Batch    2/107   train_loss = 1.108
Epoch  54 Batch    4/107   train_loss = 1.169
Epoch  54 Batch    6/107   train_loss = 1.142
Epoch  54 Batch    8/107   train_loss = 1.251
Epoch  54 Batch   10/107   train_loss = 1.197
Epoch  54 Batch   12/107   train_loss = 1.130
Epoch  54 Batch   14/107   train_loss = 1.115
Epoch  54 Batch   16/107   train_loss = 1.148
Epoch  54 Batch   18/107   train_loss = 1.135
Epoch  54 Batch   20/107   train_loss = 1.097
Epoch  54 Batch   22/107   train_loss = 1.154
Epoch  54 Batch   24/107   train_loss = 1.202
Epoch  54 Batch   26/107   train_loss = 1.116
Epoch  54 Batch   28/107   train_loss = 1.193
Epoch  54 Batch   30/107   train_loss = 1.150
Epoch  54 Batch   32/107   train_l

Epoch  57 Batch   29/107   train_loss = 1.131
Epoch  57 Batch   31/107   train_loss = 1.175
Epoch  57 Batch   33/107   train_loss = 1.023
Epoch  57 Batch   35/107   train_loss = 1.076
Epoch  57 Batch   37/107   train_loss = 1.179
Epoch  57 Batch   39/107   train_loss = 1.045
Epoch  57 Batch   41/107   train_loss = 0.983
Epoch  57 Batch   43/107   train_loss = 1.059
Epoch  57 Batch   45/107   train_loss = 0.999
Epoch  57 Batch   47/107   train_loss = 1.106
Epoch  57 Batch   49/107   train_loss = 1.131
Epoch  57 Batch   51/107   train_loss = 1.175
Epoch  57 Batch   53/107   train_loss = 1.034
Epoch  57 Batch   55/107   train_loss = 1.043
Epoch  57 Batch   57/107   train_loss = 1.122
Epoch  57 Batch   59/107   train_loss = 1.086
Epoch  57 Batch   61/107   train_loss = 1.102
Epoch  57 Batch   63/107   train_loss = 1.034
Epoch  57 Batch   65/107   train_loss = 1.025
Epoch  57 Batch   67/107   train_loss = 1.052
Epoch  57 Batch   69/107   train_loss = 1.032
Epoch  57 Batch   71/107   train_l

Epoch  60 Batch   68/107   train_loss = 0.986
Epoch  60 Batch   70/107   train_loss = 0.994
Epoch  60 Batch   72/107   train_loss = 0.967
Epoch  60 Batch   74/107   train_loss = 1.018
Epoch  60 Batch   76/107   train_loss = 0.974
Epoch  60 Batch   78/107   train_loss = 1.013
Epoch  60 Batch   80/107   train_loss = 1.065
Epoch  60 Batch   82/107   train_loss = 1.021
Epoch  60 Batch   84/107   train_loss = 1.069
Epoch  60 Batch   86/107   train_loss = 1.043
Epoch  60 Batch   88/107   train_loss = 1.120
Epoch  60 Batch   90/107   train_loss = 0.922
Epoch  60 Batch   92/107   train_loss = 0.946
Epoch  60 Batch   94/107   train_loss = 1.000
Epoch  60 Batch   96/107   train_loss = 1.022
Epoch  60 Batch   98/107   train_loss = 0.995
Epoch  60 Batch  100/107   train_loss = 0.924
Epoch  60 Batch  102/107   train_loss = 1.046
Epoch  60 Batch  104/107   train_loss = 0.949
Epoch  60 Batch  106/107   train_loss = 0.922
Epoch  61 Batch    1/107   train_loss = 1.008
Epoch  61 Batch    3/107   train_l

Epoch  64 Batch    0/107   train_loss = 0.960
Epoch  64 Batch    2/107   train_loss = 0.897
Epoch  64 Batch    4/107   train_loss = 0.968
Epoch  64 Batch    6/107   train_loss = 0.969
Epoch  64 Batch    8/107   train_loss = 1.067
Epoch  64 Batch   10/107   train_loss = 0.981
Epoch  64 Batch   12/107   train_loss = 0.925
Epoch  64 Batch   14/107   train_loss = 0.943
Epoch  64 Batch   16/107   train_loss = 0.973
Epoch  64 Batch   18/107   train_loss = 1.005
Epoch  64 Batch   20/107   train_loss = 0.911
Epoch  64 Batch   22/107   train_loss = 0.997
Epoch  64 Batch   24/107   train_loss = 1.036
Epoch  64 Batch   26/107   train_loss = 0.974
Epoch  64 Batch   28/107   train_loss = 0.956
Epoch  64 Batch   30/107   train_loss = 0.998
Epoch  64 Batch   32/107   train_loss = 0.949
Epoch  64 Batch   34/107   train_loss = 0.941
Epoch  64 Batch   36/107   train_loss = 0.974
Epoch  64 Batch   38/107   train_loss = 0.921
Epoch  64 Batch   40/107   train_loss = 1.022
Epoch  64 Batch   42/107   train_l

Epoch  67 Batch   39/107   train_loss = 0.912
Epoch  67 Batch   41/107   train_loss = 0.922
Epoch  67 Batch   43/107   train_loss = 0.928
Epoch  67 Batch   45/107   train_loss = 0.910
Epoch  67 Batch   47/107   train_loss = 0.962
Epoch  67 Batch   49/107   train_loss = 1.026
Epoch  67 Batch   51/107   train_loss = 1.042
Epoch  67 Batch   53/107   train_loss = 0.923
Epoch  67 Batch   55/107   train_loss = 0.956
Epoch  67 Batch   57/107   train_loss = 1.055
Epoch  67 Batch   59/107   train_loss = 0.998
Epoch  67 Batch   61/107   train_loss = 0.987
Epoch  67 Batch   63/107   train_loss = 0.896
Epoch  67 Batch   65/107   train_loss = 0.946
Epoch  67 Batch   67/107   train_loss = 0.974
Epoch  67 Batch   69/107   train_loss = 0.934
Epoch  67 Batch   71/107   train_loss = 0.940
Epoch  67 Batch   73/107   train_loss = 0.946
Epoch  67 Batch   75/107   train_loss = 0.929
Epoch  67 Batch   77/107   train_loss = 0.918
Epoch  67 Batch   79/107   train_loss = 0.971
Epoch  67 Batch   81/107   train_l

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [27]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [28]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [29]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    input_ = loaded_graph.get_tensor_by_name("input:0")
    initial_state = loaded_graph.get_tensor_by_name("initial_state:0")
    final_state= loaded_graph.get_tensor_by_name("final_state:0")
    probabilities = loaded_graph.get_tensor_by_name("probs:0")
    return input_, initial_state, final_state, probabilities


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [30]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    next_word_index = np.argmax(probabilities)
    return int_to_vocab[next_word_index]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [31]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
moe_szyslak:(singing) breaking us to stick this town. it was practically everywhere three, and he is this to drink?
marge_simpson:(singing) hello it, use" delete. the the man.
moe_szyslak:(pouring) did am right? i am sick there he's street.
carl_carlson: i know a chance of raccoons your pool. i'm not here for?
barney_gumble: you know, moe, i got is the you'll a day. i'd kill for something later?
moe_szyslak:(lost to homer) but i think i do?
lenny_leonard: uh, yeah. and i know the lisa could daddy is like it.
homer_simpson:(rolls down) hey moe, listen, moe.
moe_szyslak: ah, life, girl-bart, huh?
moe_szyslak: yeah, i know, i know, but a big bucks for a man with a positive.
homer_simpson: shut, homer.
lenny_leonard: yeah, someone else do with it?
moe_szyslak: if you're gonna have to sell the chief of it.
homer_simpson: that was a


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckily there's more data!  As we mentioned in the beggining of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.